In [1]:
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")
using PyCall
using Distributions
unshift!(PyVector(pyimport("sys")["path"]), "")
unshift!(PyVector(pyimport("sys")["path"]), "..")
unshift!(PyVector(pyimport("sys")["path"]), "../Ad05RTheta1RSteer05Abs0FR200")
unshift!(PyVector(pyimport("sys")["path"]), "../CommonFiles")
@pyimport python2juliaReuse as p2j

In [2]:
# policies = ["Baseline/Rewards/b_15e3_3","Retrain/Rewards/rarl","Retrain/Rewards/rarlP10",
#             "FSP/Rewards/nBnoTR","FSP/Rewards/nBnoTRP10"]

In [3]:
pc = p2j.PolicyContainer()
path = "Data/Sep1/Ad05RTheta1RSteer05Abs0FR200/"
# policies = readdir(string(path,"Policy"))
# pnames = []
# for i = 1:length(policies)
#     if policies[i] != ".DS_Store"
#         push!(pnames,policies[i][1:length(policies[i])-4])
#     end
# end
# println(pnames)
pnames = ["b_10e3","rarl_1e3","fsp_1e3"]
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")


2018-09-05 01:57:34.540211: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-05 01:57:34.540230: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-09-05 01:57:34.540233: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-05 01:57:34.540236: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
05-Sep 01:57:34:WARNING:root:replacing docs for 'close :: Tuple{ZMQTransport}' in module 'Main'.
05-Sep 01:57:34

run_env_server_rarl (generic function with 1 method)

In [4]:
mutable struct Counter
    i::Int
    I::Int
    num_collision::Int
    rate_collision::Float64
    total_reward::Float64
    single_reward::Float64
    avg_reward::Float64
    max_reward::Float64
    min_reward::Float64
    rewards::Array{Float64}
    done::Bool
    direction::Int
    total_speed::Float64
    single_speed::Float64
    avg_speed::Float64
    speeds::Array{Float64}
    single_time::Float64
    total_time::Float64
    time_between_collisions::Float64
    total_reward_d::Float64
    single_reward_d::Float64
    avg_reward_d::Float64
    rewards_d::Array{Float64}
end
function Counter(N::Int)
    return Counter(0,N,0,0.0,0.0,0.0,0.0,-Inf,Inf,zeros(N),false,-1,
        0.0,0.0,0.0,zeros(N),0.0,0.0,0.0,0.0,0.0,0.0,zeros(N))
end

Counter

In [5]:
function handle_action!(pc,scene,models,roadway,counter,rarl)
    state = get_observation(scene,models,roadway)
    if rarl
#         action1_arr = pc[:getAction_just](state)
        dist = pc[:getActionDistribution_just](state)
        action1_arr = dist["mean"]
    else
        action1_arr = pc[:getAction_just_baseline](state)
    end
    action1_arr = tanh.(action1_arr)

    dacc = 2*rand()-2.0
    dsteer = 2*rand()-2.0
    action = Egoaction(action1_arr[1],action1_arr[2],dacc,dsteer)
    
    done, bump = simulate_action!(action,scene,models,roadway)
    counter.single_time += TIMESTEP
    counter.total_time += TIMESTEP
    reward = reward_fn(action,done,scene,models,roadway)
    if reward == -5.0
        counter.num_collision += 1
    else
        counter.total_reward_d += reward
        counter.single_reward_d += reward
    end
    
    counter.total_reward += reward
    counter.single_reward += reward
    
    speed = scene[1].state.v
    counter.single_speed += speed
    counter.total_speed += speed
    
    if done
        pc[:resetPolicy](1)
        rand_ego!(scene,models,roadway)
        
        counter.i += 1
        if counter.single_reward > counter.max_reward
            counter.max_reward = counter.single_reward
        end
        if counter.single_reward < counter.min_reward
            counter.min_reward = counter.single_reward
        end
        if counter.i > counter.I
            counter.done = true
            counter.avg_reward = counter.total_reward/counter.I
            counter.rate_collision = counter.num_collision/counter.I
            counter.avg_reward_d = counter.total_reward_d/counter.I
            counter.avg_speed = counter.total_speed/(counter.total_time/TIMESTEP)
            counter.time_between_collisions = counter.total_time/counter.num_collision
        else
            counter.rewards[counter.i] = counter.single_reward
            counter.rewards_d[counter.i] = counter.single_reward_d
            counter.speeds[counter.i] = counter.single_speed/(counter.single_time/TIMESTEP)
        end
        counter.single_reward = 0.0
        counter.single_reward_d = 0.0
        counter.single_speed = 0.0
        counter.single_time = 0.0
    end
    
    
end

handle_action! (generic function with 1 method)

In [7]:
alphas = 1.0:0.1:3.0
log_path = string(path,"AxisLength/")
scene, models, roadway = initialize_env()
for alpha in alphas
    println("alpha: ",alpha)
    for pname in pnames
        println(pname)
        policy_path = string(path,"Policy/",pname,".pkl")
        pc[:reset_policy](policy_path=policy_path)
        N = 5

        counter = Counter(N)
        rand_ego!(scene,models,roadway,L=alpha)
        pc[:resetPolicy](1)
        while !counter.done
            handle_action!(pc,scene,models,roadway,counter,true)
        end
        println(counter.avg_reward)
        println(counter.rate_collision)

        writedlm(string(log_path,pname,"_rewards_axis",alpha,".txt"),counter.rewards)
        writedlm(string(log_path,pname,"_rewards_d_axis",alpha,".txt"),counter.rewards_d)
        writedlm(string(log_path,pname,"_speeds_axis",alpha,".txt"),counter.speeds)
        writedlm(string(log_path,pname,"_collision_rate_axis",alpha,".txt"),counter.rate_collision)
#         writedlm(string(log_path,pname,"_time_between_collisions_pareto",alpha,".txt"),(counter.total_time+counter2.total_time)/(counter.num_collision+counter2.num_collision))
    end
end

alpha: 1.0
b_10e3
504.0980015607819
0.0
rarl_1e3
478.29336933888453
0.0
fsp_1e3
199.74728189343938
0.0
alpha: 1.1
b_10e3
495.53348216534306
0.0
rarl_1e3
509.1637391518664
0.0
fsp_1e3
284.5812656981542
0.0
alpha: 1.2
b_10e3
496.4048888815295
0.0
rarl_1e3
522.9974300469795
0.0
fsp_1e3
272.4968149478348
0.0
alpha: 1.3
b_10e3
494.65750756296893
0.0
rarl_1e3
502.61554234938467
0.0
fsp_1e3
209.6979390118608
0.0
alpha: 1.4
b_10e3
495.3384065125779
0.0
rarl_1e3
497.47904268926413
0.0
fsp_1e3
110.7533471772305
0.0
alpha: 1.5
b_10e3
500.1701540066636
0.0
rarl_1e3
498.16845284128186
0.0
fsp_1e3
259.89682225416925
0.0
alpha: 1.6
b_10e3
493.8933013110458
0.0
rarl_1e3
511.0345828095945
0.0
fsp_1e3
221.45598399583696
0.0
alpha: 1.7
b_10e3
504.142810515375
0.0
rarl_1e3
509.499130757531
0.0
fsp_1e3
327.67343166130945
0.0
alpha: 1.8
b_10e3
503.5050226518796
0.0
rarl_1e3
495.6577298506535
0.0
fsp_1e3
222.17454156254948
0.0
alpha: 1.9
b_10e3
480.41567601121324
0.0
rarl_1e3
502.9761400329018
0.0
fsp_1e3
18